In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("proj1.ipynb")

# Project 1 - SQL
## Due Date: Wednesday, February 14th, 5:00pm

In this project, we will be working with SQL on the IMDB database.

## Objectives

- Explore and extract relevant information from database with SQL functions
- Perform data cleaning and transformation using string functions and regex
- Use the cleaned data to run insightful analysis using joins and aggregations functions

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel, clear output, and simply re-run the notebook as normal. This will shutdown your current connection to the database, which will prevent the issue of multiple connections to the database at any given point. When re-running the notebook, you will create a fresh database based on the provided Postgres dump.

If you face slow kernel times or are unable to open your notebook, restart your server by going to File -> Hub Control Panel -> Stop My Server and then clicking Start My Server.

## Logistics & Scoring Breakdown

Please read the submission instructions carefully and double check that your submission is not throwing any errors. Please ensure that public tests pass upon submission. It is your responsibility to wait until the autograder finishes running. We will not be accepting regrade requests for submission issues.

Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests. **Free-response questions (marked 'm' in the table below) are manually graded.**

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.
|Question|Points|
|---|---|
|0|1|
|1a|1|
|1b|2|
|1c|1|
|1d|1|
|2a|1|
|2b|3|
|2c|3|
|3a|2|
|3b|2|
|3c|2|
|3d|m: 1|
|4|2|
|**Total**|22|

**Summary:** 22 points (autograded: 21, manual: 1) 

In [ ]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Before You Start: Assignment Tips


<p><b>Please Read!!</b> In this project we will assume you have attended lecture and seen how to connect to a Postgres server via two ways: JupySQL in Jupyter Notebook, and the psql command-line program (CLI).</p>

    
We have written up these instructions for you in the <a href="https://www.data101.org/sp24/resources/assignment-tips/">Spring 2024 Assignment Tips</a>—a handy resource that has many other tips that we **highly recommend** taking a moment to read through to save you time and improve your workflow:

* PostgreSQL documentation
* JupySQL and magic commands in Jupyter
* JupyterHub keyboard shortcuts
* psql and common meta-commands
* Debugging:
    * Where to create new cells to play nice with the autograder
    * Opening/closing connections, deleting databases if all else fails
* Local installation (not supported by staff officially, but for your reference)

In Jupyter Notebooks, a [cell 'magic' command](https://ipython.readthedocs.io/en/stable/interactive/magics.html) is a special command that is preceded by two percentage signs (%%). Cell magics operate on entire cells and are used to change the behavior of the entire cell. They are not part of the Python language itself but are specific to the Jupyter environment. They help us do a lot of cool things, like run SQL commands directly within Jupyter! For some questions with cell magic, we will be saving the literal query string with [query snippets](https://jupysql.ploomber.io/en/latest/api/magic-snippets.html) using `--save`, as illustrated below:

```
%%sql --save query_result << 
SELECT * FROM table ...
```

# Database Setup
We are going to be using the `JupySQL` library to connect our notebook to a PostgreSQL database server on your JupyterHub account. Running the next cell will do so; you should not see any error messages after it executes.

In [ ]:
# The first time you are running this cell, you may need to run the following line as: %load_ext sql 
%reload_ext sql

In the next cell, we will unzip the data. This only needs to be done once.

In [ ]:
!unzip -u data/imdbdb.zip -d data/

<br/>

**Create the `imdb` database**: <br>
We will use PostgreSQL commands to create a database and import our data into it. Run the following cell to do this.
* You can also run these cells in the command-line via `psql`.
* If you run into the **role does not exist** error, feel free to ignore it. It does not affect data import.

In [ ]:
!psql postgresql://jovyan@127.0.0.1:5432/imdb -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database()  AND pid <> pg_backend_pid();'
!psql postgresql://jovyan@127.0.0.1:5432/postgres -c 'DROP DATABASE IF EXISTS imdb'
!psql postgresql://jovyan@127.0.0.1:5432/postgres -c 'CREATE DATABASE imdb'
!psql postgresql://jovyan@127.0.0.1:5432/imdb -f data/imdbdb.sql

**Connect to `imdb` database in the Notebook**: 
<br>
Now let's connect to the new database we just created! There should be no errors after running the following cell.

In [ ]:
%sql postgresql://jovyan@127.0.0.1:5432/imdb

**Connect to `imdb` database in `psql`**: 

<div class="alert alert-block alert-info">
<b>Do the following in a Terminal window!</b> (To open a Terminal in DataHub, go to File -> New -> Terminal.)
</div>

Connect to the same database via `psql`. See the [Spring 2024 Assignment Tips](https://www.data101.org/sp24/resources/assignment-tips/#postgresql-client-cli) website resource for details on connecting. Being able to read and implement written instructions is important and crucial in working as a data engineer! To connect to the database, you should be typing in a command into your JupyterHub Terminal. Once successfully connected, you should see ``imdb=# ``

Once connected, run the following meta-command in the `psql` client:

``
\l
``

This should display all databases on this server, including the `imdb` database you just created.


---

**Quick check**: To make sure things are working, let's fetch 10 rows from one of our tables `cast_sample`. Just run the following cell, no further action is needed.

In [ ]:
%%sql
SELECT * 
  FROM cast_sample
LIMIT 10

## Connect to the grader

In [ ]:
# Connecting the grader
# Just run the following cell, no further action is needed.
from data101_utils import GradingUtil
grading_util = GradingUtil("proj1")
grading_util.prepare_autograder()

## The `imdb` Database

In this project, we are working with a reduced version of the Internet Movie Database (IMDb) database. This Postgres database is a small random sample of actors from the much larger full database (which is over several GBs large) and includes their corresponding movies and cast info. Disclaimer: as a result, we may obtain wildly  different results than if we were to use the entire database. 

Below is a list of the relations in our database. Please carefully take the time to understand the references between each relations, as this will be important when querying from these relations.  
- **actor_sample**: information about the actors including id, name, and gender
- **cast_sample**: each person on the cast of each movie gets a row including cast id, each person's id (`actor_sample.id`), movie id (`movie_sample.id`), and role id
- **movie_sample**: sample of movies the actors have been in, including movie id, title, and the production year
- **movie_info_sample**: this table originally had a lot of information for each movie (take a look at info_type to see the information available), but we have dropped some information to make it easier to manage. This table includes movie info's id, movie id, info type id, and the info itself
- **info_type**: reference table to match each info type id to the description of the type of information
- **role_type**: reference table for cast_sample to match role id to the description of the role

Here's an exercise for you to think about: what meta-command (in terminal/CLI) returns this bolded list?

### Key Notes
- This database is **not** the same as the IMDb lecture database, but has a lot of of similar features. 
- Point of confusion: `movie_sample` and `actor_sample` both have attributes `id` corresponding to 7 digit unique numeric identifiers, but do **not** refer to the same data values.
- `cast_sample` is analagous to the `crew` table from lecture. It can be used to match an actor's id to movies they have acted in, among other relations.
- You are highly encouraged to spend some time exploring the metadata of these tables using Postgres meta-commands to better understand the data given and the relations between tables. To get started, try outputting the schema of the tables.
- You may create unlimited CTEs, but **please do not create intermediate views** unless explicitly asked to because they may take up significant space and use up your allocated memory.

<br/><br/><br/>

<hr style="border: 1px solid #fdb515;" />

# The `information_schema` schema

A **schema** is a namespace of tables in the database, often used for security purposes. Let's see how many schema are defined for us in our current database:


In [ ]:
%%sql
SELECT * 
FROM information_schema.schemata;

Within a Postgres database, there are often at least three schemas:
* `public`, a public schema that users can access and create tables in;
* `pg_catalog`, a schema for maintaining system information; and
* `information_schema`, a schema that maintains metadata about objects currently created in the database.
* The fourth schema `pg_toast` maintains data that can't regularly be stored in relations, such as very large data values. See more in documentation [here](https://www.postgresql.org/docs/current/storage-toast.html).

For now, we focus on the `information_schema` schemata, which stores our metadata. That’s right—metadata is also data, and as we make updates to our `public` databases, metadata is automatically stored and updated into different tables under the `information_schema` schema.

There are many metadata tables that Postgres updates for us, and the full list is in the Postgres documentation [(Chapter 37)](https://www.postgresql.org/docs/current/information-schema.html). 
For now, let’s look at which the `.tables` table ([37.54](https://www.postgresql.org/docs/current/infoschema-tables.html)), which lists all the tables located in the database. Let’s specifically look at those that are in the public schema (i.e., publicly accessible tables):

In [ ]:
%%sql
SELECT * 
FROM information_schema.tables
WHERE table_schema = 'public';

<br/><br/><br/>

<hr style="border: 1px solid #fdb515;" />


# Question 0


As stated above, there are many metadata tables stored in the `information_schema` schema. Write a query that returns the names of all relations in the PostgreSQL `information_schema` schema, i.e., the names of all the metadata. Your query should look very similar to the query above where we looked at tables in the `public` schema.

**Hints:**
* Your resulting table names should correspond to what’s listed in the information schema documentation [(Chapter 37)](https://www.postgresql.org/docs/15/information-schema.html). 
* For you to think about: Why might there be fewer tables in your query response than the full list in the documentation? 

In [ ]:
%%sql --save query_0 result_0 <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_0 = %sqlcmd snippets query_0
grading_util.save_results("result_0", query_0, result_0)
result_0

In [ ]:
grader.check("q0")

<br/><br/><br/>

<hr style="border: 1px solid #fdb515;" />


# Question 1: Exploratory Data Analysis
One of the first things you'll want to do with a database table is get a sense for its metadata: column names and types, and number of rows. 

## Tutorial 

We can use the PostgreSQL `\d` meta-command to get a description of the `movie_info_sample` schema. Open up a terminal window, connect to the imdb server, and analyze the output of the meta-command: ``\d movie_info_sample``. This should be very similar to what you did in the Database Setup section near the top of this project notebook.


There are four attributes in this schema, of which `"id"` is one. What are the other attribute names? Assign `result_1a` to a list of strings, where each element is an attribute name. The list does not need to be in order.

In [ ]:
result_1a = ["id", ..., ..., ...]
result_1a

In [ ]:
grader.check("q1a")

**Debugging tip**: Throughout this project and when working with databases, you should always be checking schemas via the `\d` psql metacommand.

<br><br>

---

## Question 1b

Next, let’s continue with our initial exploration of this table. How many rows are in this table? 

Assign `result_1b` to the result of a SQL query to calculate the number of rows in the `movie_info_sample` table. Then, assign `count_1b` to the integer number of rows based on what you found in `result_1b`. Do not hard code this value.

**Hints:**
- See the [Assignment Tips](https://fa23.data101.org/resources/assignment-tips/) page for how to use SQL line magic. Note: SQL line magic is *different* from SQL multi-cell magic, which uses `%%` instead of the single `%` for single line magic. 
- Your query result should have exactly one row and one attribute; the lone value in the instance should be the number of rows.
- See the `JupySQL` [documentation](https://jupysql.ploomber.io/en/latest/intro.html) for how to index into a SQL query result.

In [ ]:
result_1b = ... # replace with %sql command. It should begin with %sql ... 
count_1b = ...

# do not edit below this line
display(result_1b)
count_1b

In [ ]:
grader.check("q1b")

<br><br>

---

## Question 1c: Random table sample

Now that we know a bit about the metadata of the table, let's randomly sample rows from `movie_info_sample` to explore its contents.

Given that you know the size of the table from the previous query, **write a query that retrieves 5 tuples on expectation using the `BERNOULLI` sampling method.** That is, if we run the query multiple times, we should get 5 tuples on *average* in our resulting table. The `BERNOULLI` sampling method scans the whole table and selects individual rows independently with `p%` probability. Please see the [documentation](https://www.postgresql.org/docs/15/sql-select.html) for syntax.


**Hints:**
* Assign `p_1c` to a sampling rate that you pass into the `query_1c` f-string using Python variable substitution (i.e., double curly braces). Your formula should contain `count_1b`. Don't forget to express `p_1c` in units of percent, i.e., `p_1c = 0.03` is 0.03%!
* For a refresher on f-strings and Python variable substitution, see [this tutorial](https://www.geeksforgeeks.org/formatted-string-literals-f-strings-python/). If Python variable substitution is done correctly, we should be able to change our `p%` probability by simply reassigning `p_1c` and rerunning the query. (Please leave `p_1c` unchanged.)
* We have completed the SQL line magic for you; this references the Python f-string `query_1c` you created within a SQL query using JupySQL-specific syntax.
* Try running the SQL cell many times and see what you notice.

In [ ]:
p_1c = ...
query_1c = f"SELECT * FROM movie_info_sample TABLESAMPLE ..." # edit this query string using variable substitution and your defined p_1c

# Do not edit below this line
result_1c = %sql {{query_1c}}

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
grading_util.save_results("result_1c", query_1c, result_1c)
result_1c

In [ ]:
grader.check("q1c")

<br><br>

---

## Question 1d: Random sample, fixed number of rows

If a random number of rows is not of importance, a more efficient way to get some arbitrary tuples from a table is to use the `ORDER BY` and `LIMIT` clauses. In the next cell, fetch 5 **random** tuples from `movie_info_sample`. Compared to the previous question, your query result here should always have 5 tuples!

**Hint**: Check out lecture.


In [ ]:
%%sql --save query_1d result_1d <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_1d = %sqlcmd snippets query_1d
grading_util.save_results("result_1d", query_1d, result_1d)
result_1d

In [ ]:
grader.check("q1d")

<br/><br/><br/>
<hr style="border: 1px solid #fdb515;" />


# Question 2: Data Cleaning

The `movie_sample` table contains a very minimal amount of information per movie:

In [ ]:
%sql SELECT * FROM movie_sample LIMIT 5;

In this question, we’re going to create a nice, refined view of the `movie_sample` table that also includes a **rating field**, called `movie_ratings`.

The [MPAA rating](https://www.motionpictures.org/film-ratings/) is commonly included in most datasets about movies, including ours, but in its current format in the dataset, it’s quite difficult to extract.

The first clue about our approach comes from the random rows you explored in Question 1. As you saw, the `movie_info_sample` table contains a lot of information about each movie. Each row contains a particular type of information (e.g., runtime, languages) categorized by `info_type_id`. Based on the other tables in this database, the `info_type` table is a reference table to this 
ID number.

Our strategy in this question is therefore as follows:
* **Question 2a**: Find the `mpaa_rating_id` from the `info_type` table.
* **Question 2b**: Extract the MPAA rating of a specific movie from the `movie_info_sample` table.
* **Question 2c**: Construct a view `movie_ratings` based on the `movie_sample` table and all relevant MPAA ratings extracted from the `movie_info_sample` table.

<br><br>

---

## Question 2a: MPAA Rating and `info_type`

To start, using the `info_type` table, write a query to find which `id` corresponds to a film's MPAA rating.  The query `result_2a` that you write should return a relation with exactly one row and one attribute; the lone value in the instance should be the MPAA rating id number. We've then assigned `mpaa_rating_id` to extract the number itself from the relation.

**Hints:** 
- Open the `psql` client in a terminal to explore the schema of `info_type` via the `\d` metacommand (see the [Assignment Tips](https://fa23.data101.org/resources/assignment-tips/) page). Remember you can also write SQL commands to that terminal to interact with the IMDB database, but all final work must be submitted through this Jupyter Notebook.
- Be careful when using quotes. SQL interprets single and double quotes differently. The single quote character `'` is reserved for delimiting string constants, while the double quote `"` is used for naming tables or columns that require special characters. See [documentation](https://www.postgresql.org/docs/current/sql-syntax-lexical.html) for more.

In [ ]:
result_2a = ... # replace with %sql command
mpaa_rating_id = result_2a[0][0]

# do not edit below this line
display(result_2a)
mpaa_rating_id

In [ ]:
grader.check("q2a")

<br><br>

---

## Question 2b: Looking up the MPAA Rating

Suppose we wanted to find the MPAA rating for the 2004 American teen drama classic, _Mean Girls_. The below cell assigns `movie_id_2b` to the IMDb ID of this movie, 2109683.

In [ ]:
# Just run this cell, no further action is needed.
movie_id_2b = 2109683
%sql SELECT * FROM movie_sample WHERE id = {{movie_id_2b}};

In the next cell, **write a query to find the MPAA rating for this movie.** Your query should return a relation with exactly one row , which has `(info, mpaa_rating)`, where `info` is the full MPAA rating string from `movie_info_sample`, and `mpaa_rating` is just the rating itself (i.e. `R`, `PG-13`, `PG`, etc) for this movie.


**Before you get started**:
* Explore the `movie_info_sample` tuples corresponding to the MPAA rating by using metacommands in the terminal. The `info` field is a little longer than just the rating. It also includes an explanation for why that movie received its rating. 
* You will need to extract a substring from the `info` column of `movie_info_sample`; you can use the [string functions](https://www.postgresql.org/docs/current/functions-string.html) in PostgreSQL to do it. There are many possible solutions. One possible solution is to use the substring function along with regex. If you use this approach, [this section on regex](https://www.postgresql.org/docs/current/functions-matching.html#FUNCTIONS-POSIX-REGEXP) may be particularly useful. With your chosen approach, ensure that you are returning a single element, and not an array! You may also index into an array. [regex101.com](https://regex101.com) may additionally be helpful in crafting your regular expressions.
* You may use `mpaa_rating_id` and `movie_id_2b` directly in the rest of the questions using Python variable substitution (i.e., double curly braces). See the `JupySQL` documentation for more details.

In [ ]:
%%sql --save query_2b result_2b << 
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2b = %sqlcmd snippets query_2b
grading_util.save_results("result_2b", query_2b, result_2b)
result_2b

In [ ]:
grader.check("q2b")

You may use `mpaa_rating_id` directly in the rest of the questions using python variable substitution.

<br><br>

---

## Question 2c
In the next cell,
1. Construct a view named `movie_ratings` containing one row for each movie, which has `(movie_id, title, info, mpaa_rating)`, where `info` is the full MPAA rating string from `movie_info_sample`, and `mpaa_rating` is just the rating itself (i.e. `R`, `PG-13`, `PG`, etc).
    * In other words, extend `movie_sample` with the MPAA rating attributes that you found in the previous question part, but this time for all movies.
2. Following the view definition, also write a `SELECT` query to return the **first 20 rows** of the view, ordered by ascending `movie_id`.

In [ ]:
%%sql --save query_2c result_2c << 
DROP VIEW IF EXISTS movie_ratings;
CREATE VIEW movie_ratings AS

...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2c = %sqlcmd snippets query_2c
grading_util.save_results("result_2c", query_2c, result_2c)
result_2c

In [ ]:
grader.check("q2c")

<br/><br/><br/>

<hr style="border: 1px solid #fdb515;" />

# Question 3: Movie Moola
One measure of a movie's success is how much money it makes. If we look at our `info_type` table, we have information about the film's gross earnings and the budget for a film. It would be nice to know how much money a film made using the profit formula:
$$profit = earnings - moneyspent$$

We start by taking a look at the gross info type, with `info_type_id = 107`.

In [ ]:
%%sql
SELECT * 
FROM movie_info_sample
WHERE info_type_id = 107
ORDER BY id
LIMIT 10 OFFSET 100000;

There are a lot of things to notice here. First of all, the values in the `info` attribute are strings with not only the earnings, but also the country and the month the earnings are cummulatively summed until. Additionally, the info values are not all in the same currency! On top of that, it appears as if some of the gross earnings, even for those in USD are from worldwide sales, while others only count sales within the USA.

For consistency, let's only use movies with gross earnings counted in the USA and that are in US Dollars ($). 

<br><br>

---

## Question 3a: Earnings
We want the numerical part of the `info` column and the **maximum earnings value** for a particular film. 

In the next cell,
- Construct a view named `movie_gross` containing **one row for each movie**, which has `(gross, movie_id, title)`, where `gross` is the numeric dollar amount of the **maximum earnings value** extracted as a float.
- To take a look at our cleaned data that we just created in our `movie_gross` view, write a `SELECT` query to display the **top 10 highest grossing films** from `movie_gross`. This query should occur in the same cell and should **ONLY** list your limited top 10 output. Do not use `GROUP BY` or other aggregate functions outside of your view. 

**Hints:** 
- The way we extracted the MPAA rating is very similar to how we want to isolate the numeric dollar amount as a string. There are multiple ways of doing this.
- Look at the [documentation](https://www.postgresql.org/docs/9.4/functions-matching.html) for the `regexp_replace` function, and specifically 'flag g'.
- Again, as a reminder: we will only be using movies with gross earnings counted in the **USA** and that are in US Dollars (**$**). Ensure that your information string contains both of these markers. The information string contains reported gross earnings as (non-USA) AND (USA), so please check that you are not adding extraneous values. 
- Be careful when trying to match IDs from one relation to another relation. Take a look at the schema of the relations you are working with, whether in terminal or by referring to the description at the top of the notebook. Ensure both fields reference the same entity. For example, `movie_info_sample.id` does not refer to the same entity as `movie_sample.id`.
- Movies may have the same name! You may need to use more than 1 attribute to uniquely identify a movie.
- The staff solution found it helpful to make an additional CTE. You may make as many CTEs as you find necessary. Do **not** create views as this may take up significant space in memory. If your kernel is crashing repeatedly, check that you are not creating any views in memory.

Your output for attribute **title** should match below. Please ensure that your final output has attributes `gross, movie_id, title` for each unique movie.
|                                      title |
|-------------------------------------------:|
|                                     Avatar |
|                                    Titanic |
|                               The Avengers |
|                            The Dark Knight |
|                                  Star Wars |
|                      The Dark Knight Rises |
|                                    Shrek 2 |
|                 E.T. the Extra-Terrestrial |
|  Star Wars: Episode I - The Phantom Menace |
| Pirates of the Caribbean: Dead Man's Chest |n: Dead Man's Chest |


In [ ]:
%%sql --save query_3a result_3a <<

DROP VIEW IF EXISTS movie_gross;
CREATE VIEW movie_gross AS

...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_3a = %sqlcmd snippets query_3a
grading_util.save_results("result_3a", query_3a, result_3a)
result_3a

In [ ]:
grader.check("q3a")

<br/>

---

## Tutorial: Budget
We will now look at the budget info type, with `info_type_id = 105`.

In [ ]:
%%sql
SELECT * 
FROM movie_info_sample
WHERE info_type_id = 105
ORDER BY id
LIMIT 10 OFFSET 5000;

Similar to when we examined the gross info, we see a lot of non-US dollar currencies. For consistency, let's only use movies **with a budget in US dollars.**

## Question 3b:

Now, we want something similar for the budget of the film, so that we can perform the subtraction of `gross` and `budget`. We want the numerical part of the `info` column and the **maximum budget value** for a particular film (as you can verify, some movies have more than one budget). 

In the next cell,
- Construct a view named `movie_budget` containing one row for each movie, which has `(budget, movie_id, title)`, where `budget` is the numeric dollar amount extracted as a float.
- To take a look at our cleaned data, write a `SELECT` query to display the **top 10 highest budget films** from `movie_budget`. When multiple films have the same budget, break ties by `movie_id` (ascending).

**Hint:** The query here should be quite similar to Question 3a. Make sure to break ties properly!

In [ ]:
%%sql --save query_3b result_3b <<

DROP VIEW IF EXISTS movie_budget;
CREATE VIEW movie_budget AS

...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_3b = %sqlcmd snippets query_3b
grading_util.save_results("result_3b", query_3b, result_3b)
result_3b

In [ ]:
grader.check("q3b")

<br><br>

---

## Question 3c

We have all the parts we need to calculate the profits. Using the `movie_gross` and `movie_budget` views created above, we can now subtract the numeric columns and save the result in another column called `profit`.

In the next cell, construct a view named `movie_profit` containing one row for each movie, which has `(movie_id, title, profit)`, where `profit` is the result of subtracting that movie's `budget` from `gross`. Following the view definition, write a `SELECT` query to return the **first 10 rows** of the view ordered by descending `profit`. This may take a while to execute.

In [ ]:
%%sql --save query_3c result_3c <<

DROP VIEW IF EXISTS movie_profit;
CREATE VIEW movie_profit AS

...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_3c = %sqlcmd snippets query_3c
grading_util.save_results("result_3c", query_3c, result_3c)
result_3c

In [ ]:
grader.check("q3c")

<!-- BEGIN QUESTION -->

<br><br>

---

## Question 3d

We analyzed the data, but something seems odd. Upon closer look, there are many negative values for `profit`. For example, the movie `102 Dalmatians` looks to have lost around $18M, but it was a widely successful film! What may account for this issue? Think about how we constrained our data from the start of the problem.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 1px solid #fdb515;" />

# Question 4: Joins

Joins are a powerful tool in database cleaning and analysis. They allow for the user to create useful tables and bring together information in a meaningful way. 

There are many types of joins: inner, outer, left, right, etc. Let's practice these in a special scenario. 

You are now working as a talent director and you need a list of all people who have been in the role `actor` (**NOT** `actress`) and the number of movies in which they have acted. Of note, the original IMDB data maintains an antiquated gender binary in their encoding of roles that is not necessarily reflective of individuals' true genders.

- Create a view called `number_movies`, which has columns `id`, `name`, `number` where `id` is the actor's id, `name` is the actor's name, and `number` is the number of movies they have acted in.
- Following your view, write a ``SELECT`` query to display the **top 10 actors** who have been in the most films. Do not include actresses.

**Note:** The `cast_sample` may include actors not included in `actor_sample` table. We still want to include these actors in our result by reference to their id. The `name` field can be NULL.

In [ ]:
%%sql --save query_4 result_4 <<

DROP VIEW IF EXISTS number_movies;
CREATE VIEW number_movies AS 

...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4 = %sqlcmd snippets query_4
grading_util.save_results("result_4", query_4, result_4)
result_4

In [ ]:
grader.check("q4")

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Congratulations! You have finished Project 1.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**For submission on Gradescope, you will need to submit the `proj1.zip` file generated by the export cell.** Please ensure that your submission includes `proj1.pdf`. 

**Please ensure that public tests pass upon submission.** It is your responsibility to wait until the autograder finishes running. We will not be accepting regrade requests for submission issues.

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab.

In [ ]:
grading_util.prepare_submission_and_cleanup()

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(files=['results.zip'])